# E-boekhouden

In [ ]:
import os
from dataclasses import dataclass

from zeep import Client
from zeep.helpers import serialize_object
import pandas as pd

### Required credentials:

In [ ]:
server_url = "https://soap.e-boekhouden.nl/soap.asmx?wsdl"

In [ ]:
username = "<your_username_here>"
security_code_1 = "<your_security_code_1_here>"
security_code_2 = "<your_security_code_1_here>"

username = os.environ.get("EBOEKHOUDEN_USERNAME")
security_code_1 = os.environ.get("EBOEKHOUDEN_SECURITY1")
security_code_2 = os.environ.get("EBOEKHOUDEN_SECURITY2")

### Create session

In [ ]:
client = Client(server_url)

In [ ]:
response_object = client.service.OpenSession(
    Username=username,
    SecurityCode1=security_code_1,
    SecurityCode2=security_code_2,
)
session_id = response_object.SessionID
print(session_id)

### Mutaties

In [ ]:
@dataclass
class Mutaties:
    mutatienr: int
    mutatienrvan: int
    mutatienrtm: int
    factuurnummer: str
    datumvan: str
    datumtm: str

    def export(self):
        return dict(
            MutatieNr=self.mutatienr,
            MutatieNrVan=self.mutatienrvan,
            MutatieNrTm=self.mutatienrtm,
            Factuurnummer=self.factuurnummer,
            DatumVan=self.datumvan,
            DatumTm=self.datumtm,
        )

In [ ]:
mutatie_filter = Mutaties(
    0, 
    0, 
    99999, 
    "", 
    "2022-01-01", 
    "2023-03-31",
).export()

In [ ]:
mutaties = client.service.GetMutaties(
    SessionID=session_id, 
    SecurityCode2=security_code_2, 
    cFilter=mutatie_filter,
)

In [ ]:
rows = []
for mutatie in mutaties.Mutaties.cMutatieList:
    flat_mutatie = dict(serialize_object(mutatie))
    del flat_mutatie['MutatieRegels']
    for regel in mutatie['MutatieRegels']['cMutatieListRegel']:
        flat_regel = dict(serialize_object(regel))
        new_mutatie = flat_mutatie.copy() | flat_regel
        rows.append(new_mutatie)

In [ ]:
df = pd.DataFrame(rows)

In [ ]:
df.to_csv('mutaties.csv')

In [ ]:
df